# Quark Mass Analysis in TSQVT

This notebook demonstrates the quark mass fitting pipeline for the Twistorial Spectral Quantum Vacuum Theory (TSQVT).

**Reference:** Appendix N of "The Geometric Origin of Three Fermion Generations"

**Repository:** https://github.com/KerymMacryn/three-generations-repo

## Contents
1. Introduction and Setup
2. Experimental Data
3. TSQVT Mass Model
4. Frequentist Fit
5. Bayesian Analysis (MCMC)
6. Fine-Tuning Diagnostics
7. Visualization

## 1. Introduction and Setup

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Import our module
import sys
sys.path.insert(0, '../scripts')
from quark_fits import (
    QuarkFitter, get_mass_vector, get_covariance_matrix,
    params_to_physical, physical_to_params, predict_masses,
    fine_tuning_matrix, global_fine_tuning_index,
    run_mcmc
)

print("Setup complete!")

## 2. Experimental Data

We use MS-bar quark masses at the Z boson mass scale from PDG 2022.

In [ ]:
# Load experimental data
m_obs = get_mass_vector('mZ')
Sigma = get_covariance_matrix('mZ', correlations=False)
sigma = np.sqrt(np.diag(Sigma))

# Display data
quark_names = ['u', 'c', 't', 'd', 's', 'b']
print("Quark masses at μ = m_Z (GeV):")
print("-" * 40)
for i, name in enumerate(quark_names):
    print(f"  m_{name} = {m_obs[i]:.6f} ± {sigma[i]:.6f}")

# Visualize mass hierarchy
fig, ax = plt.subplots(figsize=(10, 5))
colors = ['#e74c3c', '#e74c3c', '#e74c3c', '#3498db', '#3498db', '#3498db']
ax.bar(quark_names, np.log10(m_obs), color=colors, alpha=0.7, edgecolor='black')
ax.set_ylabel('log₁₀(mass / GeV)')
ax.set_title('Quark Mass Hierarchy')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
for i, (name, m) in enumerate(zip(quark_names, m_obs)):
    ax.annotate(f'{m:.2e}', (i, np.log10(m) + 0.3), ha='center', fontsize=9)
plt.tight_layout()
plt.show()

## 3. TSQVT Mass Model

In TSQVT, quark masses arise from vacuum condensation:

$$m_i^{(u)} = y_i^{(u)} \rho_i, \qquad m_i^{(d)} = y_i^{(d)} \rho_i$$

where:
- $\rho_i$ are vacuum condensation values (common to up and down sectors)
- $y_i^{(u,d)}$ are Yukawa couplings
- Index $i = 1, 2, 3$ labels generations

The ordering constraint $\rho_1 < \rho_2 < \rho_3$ is enforced via sigmoid reparameterization.

In [ ]:
# Demonstrate the reparameterization
def sigmoid(x):
    return np.where(x >= 0, 1/(1+np.exp(-x)), np.exp(x)/(1+np.exp(x)))

# Example: unconstrained parameters → physical parameters
theta_example = np.array([
    -3.0, -1.0, 2.0,       # s1, s2, s3 → rho
    -6.0, 0.0, 5.0,        # eta_u → y_u
    -5.0, -2.0, 1.5        # eta_d → y_d
])

rho, y_u, y_d = params_to_physical(theta_example)

print("Example parameter transformation:")
print(f"  ρ₁ = {rho[0]:.6f}")
print(f"  ρ₂ = {rho[1]:.6f}")
print(f"  ρ₃ = {rho[2]:.6f}")
print(f"  Ordering satisfied: {rho[0] < rho[1] < rho[2]}")
print()
print(f"  y_u = {y_u}")
print(f"  y_d = {y_d}")

## 4. Frequentist Fit

We minimize the chi-square:

$$\chi^2(\Theta) = (\mathbf{m}^{\mathrm{obs}} - \mathbf{m}^{\mathrm{mod}})^T \Sigma^{-1} (\mathbf{m}^{\mathrm{obs}} - \mathbf{m}^{\mathrm{mod}})$$

In [ ]:
# Create fitter and run optimization
fitter = QuarkFitter(scale='mZ', correlations=False)

print("Running optimization with multiple random starts...")
result = fitter.fit(method='L-BFGS-B', n_starts=20, verbose=True)

In [ ]:
# Display results
print(fitter.summary())

In [ ]:
# Visualize fit quality
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Predicted vs Observed
ax = axes[0]
m_pred = result['m_pred']
m_obs = result['m_obs']

# Use log scale for better visualization
ax.errorbar(np.log10(m_obs), np.log10(m_pred), 
            xerr=sigma/(m_obs * np.log(10)),
            fmt='o', markersize=10, capsize=5, color='#2c3e50')

# Perfect fit line
lims = [-6, 3]
ax.plot(lims, lims, 'r--', label='Perfect fit', alpha=0.7)

# Labels
for i, name in enumerate(quark_names):
    ax.annotate(name, (np.log10(m_obs[i]) + 0.15, np.log10(m_pred[i]) + 0.15),
                fontsize=12, fontweight='bold')

ax.set_xlabel('log₁₀(m_obs / GeV)')
ax.set_ylabel('log₁₀(m_pred / GeV)')
ax.set_title('TSQVT Fit: Predicted vs Observed Masses')
ax.legend()
ax.set_xlim(lims)
ax.set_ylim(lims)

# Right: Residuals
ax = axes[1]
residuals = (m_obs - m_pred) / sigma
colors = ['#e74c3c' if r > 0 else '#3498db' for r in residuals]
ax.barh(quark_names, residuals, color=colors, alpha=0.7, edgecolor='black')
ax.axvline(x=0, color='black', linewidth=2)
ax.axvline(x=-1, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Pull (σ)')
ax.set_title('Fit Residuals')

plt.tight_layout()
plt.show()

## 5. Bayesian Analysis (MCMC)

For a more complete uncertainty quantification, we sample the posterior distribution using MCMC.

In [ ]:
# Check if emcee is available
try:
    import emcee
    MCMC_AVAILABLE = True
except ImportError:
    MCMC_AVAILABLE = False
    print("emcee not installed. Install with: pip install emcee")
    print("Skipping MCMC analysis.")

In [ ]:
if MCMC_AVAILABLE:
    print("Running MCMC sampling (this may take a few minutes)...")
    mcmc_result = run_mcmc(fitter, n_walkers=32, n_steps=3000, n_burn=500, progress=True)
    
    print(f"\nMCMC Results:")
    print(f"  Acceptance fraction: {mcmc_result['acceptance_fraction']:.3f}")
    print(f"\n  ρ values (mean ± std):")
    for i in range(3):
        print(f"    ρ_{i+1} = {mcmc_result['rho_mean'][i]:.6f} ± {mcmc_result['rho_std'][i]:.6f}")

In [ ]:
if MCMC_AVAILABLE:
    # Corner plot for rho parameters
    try:
        import corner
        
        fig = corner.corner(
            mcmc_result['rho_samples'],
            labels=[r'$\rho_1$', r'$\rho_2$', r'$\rho_3$'],
            quantiles=[0.16, 0.5, 0.84],
            show_titles=True,
            title_kwargs={"fontsize": 12}
        )
        fig.suptitle('Posterior Distribution of Vacuum Condensation Values', y=1.02)
        plt.show()
    except ImportError:
        print("corner package not available for visualization")
        
        # Simple histogram instead
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
        for i in range(3):
            axes[i].hist(mcmc_result['rho_samples'][:, i], bins=50, 
                        density=True, alpha=0.7, color='steelblue')
            axes[i].axvline(mcmc_result['rho_mean'][i], color='red', 
                           linestyle='--', label='Mean')
            axes[i].set_xlabel(f'ρ_{i+1}')
            axes[i].set_ylabel('Density')
            axes[i].legend()
        plt.tight_layout()
        plt.show()

## 6. Fine-Tuning Diagnostics

We compute the fine-tuning measure:

$$\Delta = \max_{i,j} \left| \frac{\partial \ln m_i}{\partial \ln \theta_j} \right|$$

A natural theory should have $\Delta \lesssim 100$.

In [ ]:
# Compute fine-tuning matrix
theta_best = result['theta']
Delta_matrix = fine_tuning_matrix(theta_best)

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

param_names = ['s₁', 's₂', 's₃', 'η_u1', 'η_u2', 'η_u3', 'η_d1', 'η_d2', 'η_d3']
im = ax.imshow(np.log10(Delta_matrix + 1e-10), cmap='YlOrRd', aspect='auto')

ax.set_xticks(range(9))
ax.set_xticklabels(param_names)
ax.set_yticks(range(6))
ax.set_yticklabels(quark_names)
ax.set_xlabel('Parameter')
ax.set_ylabel('Mass')
ax.set_title('Fine-Tuning Matrix: log₁₀|∂ln m / ∂ln θ|')

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('log₁₀(Δ)')

plt.tight_layout()
plt.show()

print(f"Global fine-tuning index: Δ = {global_fine_tuning_index(theta_best):.1f}")

## 7. Summary and Conclusions

The TSQVT framework successfully fits the six quark masses with:
- Three vacuum condensation values $\rho_1 < \rho_2 < \rho_3$
- Generation-dependent Yukawa couplings

Key results:
1. The mass hierarchy emerges naturally from the vacuum structure
2. The fit is perturbatively stable (Yukawas < 4π)
3. Fine-tuning is within acceptable bounds

In [ ]:
# Final summary table
print("="*60)
print("TSQVT QUARK MASS FIT - FINAL RESULTS")
print("="*60)
print("\nVacuum structure:")
print(f"  ρ₁ = {result['rho'][0]:.6f}  (1st generation)")
print(f"  ρ₂ = {result['rho'][1]:.6f}  (2nd generation)")
print(f"  ρ₃ = {result['rho'][2]:.6f}  (3rd generation)")
print(f"\nHierarchy ratios:")
print(f"  ρ₂/ρ₁ = {result['rho'][1]/result['rho'][0]:.1f}")
print(f"  ρ₃/ρ₂ = {result['rho'][2]/result['rho'][1]:.2f}")
print(f"\nFit quality:")
print(f"  χ² = {result['chi2']:.4f}")
print(f"  Fine-tuning Δ = {result['fine_tuning_index']:.1f}")
print("="*60)